In [126]:
from numpy import linalg as npl 
import scipy 
from scipy import interpolate
import numpy as np 
import matplotlib.pyplot as plt

def convol(a,b):
    return np.fft.ifftshift(np.fft.ifft2(np.fft.fft2(a)*np.fft.fft2(b)))


def H(x, eps=1e-7): # x = [x1,x2], H(x) = sqrt(x1^2 + x2^2 + e^2)
    return np.sqrt(x[0]**2 + x[1]**2 +eps**2)

def Hvect(x, eps=1e-7): # x.shape = [r,c,2] ; M.shape = [r,c]
    r = len(x)
    c = len(x[0])
    M = np.zeros([r,c])
    for i in range(r):
        for j in range(c):
            M[i][j] = H(x[i,j],eps)
    return M

def gradH(x,eps=1e-7):   # x = [x1,x2], gradH(x) = [diff(H,x),diff(H,y)]    
    return [x[0]/H(x,eps), x[1]/H(x,eps)] # Remark: diff(H, x) = x / H(x)

def gradHvect(x,eps=1e-7):   # x.shape = [r,c,2], gradH.shape = [r,c,2]
    r = len(x)
    c = len(x[0])
    gradH_ = np.zeros([r,c,2])
    for i in range(r):
        for j in range(c):
            gradH_[i][j] = gradH(x[i][j], eps)
    return gradH_

def F(p, eps=1e-7):
    N = len(p)  
    F = 0
    for i in range(N):
        for j in range(N):
            F += H(p[i]-p[j],eps)
    return F/(2*N**2)

def gradF(p): 
    N = len(p)
    gradF_ = np.zeros([N,2])
    for i in range(N):
        s = 0
        for j in range(i):
            s = s + 1/H(p[i]-p[j],eps)
        for j in range(i+1,N):
            s = s + 1/H(p[i]-p[j],eps)
        gradF_[i] = p[i] * s / N**2
    return gradF_


def compute_integral(pi, grid,eps = 1e-7):
    r,c = pi.shape
    r_padded = 2*r + 1
    c_padded = 2*c + 1
    pi_padded = np.zeros([r_padded,c_padded])
    padr = (r+1) // 2
    padc = (c+1) // 2
    pi_padded[padr:-padr][:,padc:-padc] = pi 
    S = grid[-1][-1] + (grid[0][1]-grid[0][0])*(r_padded-r)/2.

    #     grid.shape = r,c,2 
    #     grid_padded.shape = [r_padded, c_padded,2]
    x = np.linspace(-S[0],S[0],r_padded)
    y = np.linspace(-S[1],S[1],c_padded)
    x_padded, y_padded=  np.meshgrid(x, y)
    grid_padded = np.zeros([r_padded,c_padded,2])
    grid_padded[:,:,0] = x_padded
    grid_padded[:,:,1] = y_padded
    
    Hgrid= Hvect(grid_padded,eps)
    conv = convol(pi_padded, Hgrid)
    conv = conv[padr:-padr][:,padc:-padc]
    return conv.real

def compute_integral_prime(pi, grid): # pi.shape = (r,c), grid.shape = (r,c,2)
    
    # Output: two functions: each function is represented by a value grid of shape (r,c) -> (r,c,2)
    # Convolutions of diff(H,x) and diff(H,y) with pi 
    r,c = pi.shape
    r_padded = 2*r + 1
    c_padded = 2*c + 1
    pi_padded = np.zeros([r_padded,c_padded])
    padr = (r+1) // 2
    padc = (c+1) // 2
    pi_padded[padr:-padr][:,padc:-padc] = pi 
    S = grid[-1][-1] + (grid[0][1]-grid[0][0])*(r_padded-r)/2.

    x = np.linspace(-S[0],S[0],r_padded)
    y = np.linspace(-S[1],S[1],c_padded)
    x_padded, y_padded=  np.meshgrid(x, y)
    grid_padded = np.zeros([r_padded,c_padded,2]) # grid_padded.shape = [r_padded, c_padded,2]
    grid_padded[:,:,0] = x_padded
    grid_padded[:,:,1] = y_padded

    gradHgrid = gradHvect(grid_padded,eps) # gradHgrid.shape = (r_padded, c_padded, 2)
    conv = np.zeros([r_padded,c_padded,2])
    
    conv[:,:,0] = convol(pi_padded, gradHgrid[:,:,0]) # conv( diff(H, x) , pi)
    conv[:,:,1] = convol(pi_padded, gradHgrid[:,:,1]) # conv( diff(H, y) , pi)
    
    conv = conv[padr:-padr][:,padc:-padc]
    return conv.real

def interp(grid, val):
    return interpolate.RectBivariateSpline(grid[:,:,0][0],grid[:,:,1][:,0], conv.real)

def interp_grad(grid, val):
    return interp(grid, val[:,:,0]), interp(grid,val[:,:,1])

def G(p):
#     Input: (p1,...,pN) (matrix of size N*2) : p_x = p[:,0] ; p_y = p[:,1]
#     Output: G at (p1,...,pN) is a real number 
    num_pts = len(p)
    val = compute_integral(pi, grid)
    g = interp(grid,val)
    G = 0
    for i in range(num_pts):
        G += (g(p[i][0],p[i][1]))
    return G

def gradG(p): 
#     Input: (p1,...,pN) (matrix of size N*2) 
#     Output: gradG at (p1,...,pN) (matrix of size N*2)
    num_pts = len(p)
    val = compute_integral_prime(pi, grid)
    G1, G2 = interp_grad(grid,val)
#     gradG(p) = [G1(p1), G2(p1), G1(p2), G2(p2),..., G1(pN), G2(pN)]
    grad = np.zeros([N,2])
    for i in range(num_pts):
        grad[i,0] = G1(p[i,0],p[i,1])
        grad[i,1] = G2(p[i,0],p[i,1])
    return grad/num_pts

def J(p):
    return G(p) - F(p)

def gradJ(p):
    return gradG(p) - gradF(p)

def Gradient_descent(J,gradJ,h=1e-1, pini = np.zeros(20)): 
    p = np.copy(pini)
    y = [p]
    eps = 1e-10
    itermax = 1000
    err = 2*eps
    iter = 0
    while err>eps and iter<itermax:
        p = p - h*gradJ(p)
        y.append(p)
        err = np.linalg.norm(gradJ(p))
        iter += 1
        if iter%100==0:
            plt.scatter(p,np.zeros_like(p))
            plt.show()
    xiter=np.array(y)
    return p,xiter,iter

In [2]:
from PIL import Image
image = Image.open('gs.jpeg')
image = image.convert('L')
new_image = image.resize((249, 249))
new_image.save('image_249.jpg')

img = np.array(new_image)

In [134]:
pi = np.ones([49,49]) 
N =49
r,c = pi.shape

x = np.linspace(-0.5,0.5,N)
x_, y_=  np.meshgrid(x, x)
grid_padded = np.zeros([r,c,2])
grid_padded[:,:,0] = x_
grid_padded[:,:,1] = y_
print(gradHvect(grid ))
num_pts = 20
x_i = np.linspace(-0.2,0.2,num_pts)
y_i = np.linspace(-0.3,0.3,num_pts)
p = np.zeros([num_pts,2])
p[:,0] = x_i
p[:,1] = y_i
print(p)
print(gradG(p))

[[[-0.70710678 -0.70710678]
  [-0.69190536 -0.72198821]
  [-0.67572463 -0.73715414]
  ...
  [ 0.67572463 -0.73715414]
  [ 0.69190536 -0.72198821]
  [ 0.70710678 -0.70710678]]

 [[-0.72198821 -0.69190536]
  [-0.70710678 -0.70710678]
  [-0.69122265 -0.72264186]
  ...
  [ 0.69122265 -0.72264186]
  [ 0.70710678 -0.70710678]
  [ 0.72198821 -0.69190536]]

 [[-0.73715414 -0.67572463]
  [-0.72264186 -0.69122265]
  [-0.70710678 -0.70710678]
  ...
  [ 0.70710678 -0.70710678]
  [ 0.72264186 -0.69122265]
  [ 0.73715414 -0.67572463]]

 ...

 [[-0.73715414  0.67572463]
  [-0.72264186  0.69122265]
  [-0.70710678  0.70710678]
  ...
  [ 0.70710678  0.70710678]
  [ 0.72264186  0.69122265]
  [ 0.73715414  0.67572463]]

 [[-0.72198821  0.69190536]
  [-0.70710678  0.70710678]
  [-0.69122265  0.72264186]
  ...
  [ 0.69122265  0.72264186]
  [ 0.70710678  0.70710678]
  [ 0.72198821  0.69190536]]

 [[-0.70710678  0.70710678]
  [-0.69190536  0.72198821]
  [-0.67572463  0.73715414]
  ...
  [ 0.67572463  0.737154

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
